# Energy-Based Models (TensorFlow/Keras)    Hopfield networks, Boltzmann energy, and a minimal RBM trained with Contrastive Divergence.

In [ ]:
    # !pip install tensorflow matplotlib scikit-learn    import numpy as np    import matplotlib.pyplot as plt    import tensorflow as tf    from sklearn.datasets import load_digits    from sklearn.model_selection import train_test_split    from sklearn.preprocessing import MinMaxScaler    tf.keras.utils.set_random_seed(0)    

## Hopfield Networks    Store binary patterns and recover them from noisy cues.

In [ ]:
    def train_hopfield(patterns):        n = patterns.shape[1]        W = np.zeros((n, n))        for p in patterns:            W += np.outer(p, p)        np.fill_diagonal(W, 0)        W /= patterns.shape[0]        return W    def recall(W, pattern, steps=5):        state = pattern.copy()        for _ in range(steps):            state = np.sign(W @ state)        return state    patterns = np.array([[1, -1, 1, -1], [-1, -1, 1, 1]])    W = train_hopfield(patterns)    noisy = np.array([1, -1, -1, -1])    recalled = recall(W, noisy)    print("Noisy:", noisy)    print("Recalled:", recalled)    

## Boltzmann Machines    Compute energies for binary states with symmetric weights.

In [ ]:
    def energy(state, W, b=None):        b = np.zeros_like(state) if b is None else b        return -0.5 * state @ W @ state - b @ state    W = np.array([[0, 1.2], [1.2, 0]])    b = np.array([0.3, -0.1])    for s in [np.array([1,1]), np.array([1,-1]), np.array([-1,1]), np.array([-1,-1])]:        print(f"State {s} energy: {energy(s, W, b):.3f}")    

## Restricted Boltzmann Machines (RBM)    Minimal CD-1 training on the digits dataset.

In [ ]:
    digits = load_digits()    data = MinMaxScaler().fit_transform(digits.data).astype('float32')    X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)    class SimpleRBM:        def __init__(self, n_visible, n_hidden):            self.W = tf.Variable(tf.random.normal([n_visible, n_hidden], stddev=0.01))            self.vb = tf.Variable(tf.zeros([n_visible]))            self.hb = tf.Variable(tf.zeros([n_hidden]))        def hidden_prob(self, v):            return tf.nn.sigmoid(tf.matmul(v, self.W) + self.hb)        def visible_prob(self, h):            return tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.W)) + self.vb)        def sample_prob(self, probs):            return tf.cast(tf.random.uniform(tf.shape(probs)) < probs, tf.float32)    def train_rbm(rbm, data, epochs=8, batch_size=32, lr=0.05):        ds = tf.data.Dataset.from_tensor_slices(data).shuffle(1024).batch(batch_size)        for _ in range(epochs):            for v0 in ds:                h0_prob = rbm.hidden_prob(v0)                h0 = rbm.sample_prob(h0_prob)                v1_prob = rbm.visible_prob(h0)                h1_prob = rbm.hidden_prob(v1_prob)                dW = tf.matmul(v0, h0_prob, transpose_a=True) - tf.matmul(v1_prob, h1_prob, transpose_a=True)                dvb = tf.reduce_mean(v0 - v1_prob, axis=0)                dhb = tf.reduce_mean(h0_prob - h1_prob, axis=0)                rbm.W.assign_add(lr * dW / tf.cast(tf.shape(v0)[0], tf.float32))                rbm.vb.assign_add(lr * dvb)                rbm.hb.assign_add(lr * dhb)    rbm = SimpleRBM(n_visible=X_train.shape[1], n_hidden=128)    train_rbm(rbm, X_train, epochs=8, batch_size=32, lr=0.05)    sample = X_test[:1]    recon = rbm.visible_prob(rbm.sample_prob(rbm.hidden_prob(sample)))    recon_err = tf.reduce_mean(tf.square(sample - recon)).numpy()    print("Sample reconstruction error:", recon_err)    plt.figure(figsize=(4,2))    plt.subplot(1,2,1); plt.imshow(sample.reshape(8,8), cmap='gray'); plt.title('Original'); plt.axis('off')    plt.subplot(1,2,2); plt.imshow(recon.numpy().reshape(8,8), cmap='gray'); plt.title('Recon'); plt.axis('off')    plt.show()    

## Stacking of RBMs    Train a second RBM on the hidden activations of the first.

In [ ]:
    rbm1 = SimpleRBM(n_visible=X_train.shape[1], n_hidden=64)    train_rbm(rbm1, X_train, epochs=6, batch_size=32, lr=0.05)    h_train = rbm1.hidden_prob(X_train).numpy()    rbm2 = SimpleRBM(n_visible=h_train.shape[1], n_hidden=32)    train_rbm(rbm2, h_train, epochs=6, batch_size=32, lr=0.05)    h_test = rbm1.hidden_prob(X_test).numpy()    h2_test = rbm2.hidden_prob(h_test).numpy()    print("First RBM features shape:", h_test.shape)    print("Second RBM features shape:", h2_test.shape)    